In [2]:
import pandas as pd
import vectorbtpro as vbt
from datetime import datetime, timedelta
import pytz
import numpy as np
import pandas_ta as ta
import logging
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply
logging.basicConfig(level=logging.WARNING, format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
import multiprocessing as mp
mp.set_start_method('fork')


/Users/ericervin/opt/anaconda3/envs/datascience/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [3]:

start = "2018-01-01"
# Enter your parameters here
metric = 'total_return'

start_date = datetime(2020, 1, 1, tzinfo=pytz.utc)  # time period for analysis, must be timezone-aware
end_date = datetime.now(pytz.utc)
# end_date = datetime(2020, 1, 1, tzinfo=pytz.utc)

# The following is the number of days to look back for the analysis
time_buffer = timedelta(days=100)  # buffer before to pre-calculate SMA/EMA, best to set to max window
freq = '1h'

vbt.settings.portfolio['init_cash'] = 100_000.  # 100,000$
# vbt.settings.portfolio['fees'] = 0.0025  # 0.25%
# vbt.settings.portfolio['slippage'] = 0.0025  # 0.25%

# get binance data doing it this way allows for you to update your data rather than re-downloading it
# binance_data = vbt.BinanceData.fetch(symbols,timeframe=freq, start=start_date,end="now UTC")
# binance_data.save("binance_data.pkl")

# If you already have the data downloaded, you can load it
# binance_data = vbt.BinanceData.load("binance_data.pkl")
# binance_data = binance_data.update() if you want to update it.
#futures_path = '/Users/ericervin/Documents/Coding/data-repository/data/BTCUSDT_1m_futures.pkl'
futures_path = 'test_data/BTCUSDT_1m_futures.pkl'

If you don't have the datafile yet you can grab it from this [link](https://drive.google.com/drive/folders/1jKy2DMbBow-J5jTvPutw-j17m7Ss8R3i?usp=drive_link)

In [4]:
btc = vbt.BinanceData.load(futures_path)
df = btc.get()

In [17]:
df

,Open,High,Low,Close,Volume,Quote volume,Trade count,Taker base volume,Taker quote volume,signal
Open time,,,,,,,,,,
2021-01-01 00:00:00+00:00,28948.19,28997.16,28935.30,28991.01,143.661,4.161510e+06,1120,114.439,3.315299e+06,1
2021-01-01 00:01:00+00:00,28992.98,29045.93,28991.01,29035.18,256.280,7.437731e+06,1967,179.650,5.213713e+06,0
2021-01-01 00:02:00+00:00,29036.41,29036.97,28993.19,29016.23,102.675,2.979644e+06,987,44.060,1.278719e+06,0
2021-01-01 00:03:00+00:00,29016.23,29023.87,28995.50,29002.92,85.762,2.487845e+06,832,56.768,1.646753e+06,0
2021-01-01 00:04:00+00:00,29005.93,29019.98,29000.00,29000.00,70.246,2.037999e+06,752,46.645,1.353346e+06,0
...,...,...,...,...,...,...,...,...,...,...
2023-06-03 23:08:00+00:00,27055.50,27058.10,27055.50,27058.10,33.934,9.181214e+05,538,10.977,2.970001e+05,0
2023-06-03 23:09:00+00:00,27058.10,27060.00,27058.00,27060.00,22.295,6.032967e+05,375,18.930,5.122406e+05,0
2023-06-03 23:10:00+00:00,27060.00,27060.50,27059.90,27060.50,40.601,1.098664e+06,494,24.383,6.598049e+05,0


In [5]:
# If you don't already have the data you can download it from Binance
# get binance data doing it this way allows for you to update your data rather than re-downloading it
# binance_data = vbt.BinanceData.fetch(symbols,timeframe=freq, start=start_date,end="now UTC")
# df = binance_data.get()

# Generate Entry Signals
For this I will generate an entry signal every day at midnight. The challenge will be to build a strategy that is able to choose the proper position size for the first and subsequent entries. Then take profit if the trade is quickly profitable and work itself out of a hole if the trade turns against it. I will perform this first on a strategy for long positions, then on short positions. Then we will attempt to see if we can run the strategy for more intelligent entry points. 

In [6]:
# create a signal array the same length as the dataframe
signal = np.zeros(len(df))
signal = pd.Series(signal, index=df.index)
# set the signal to 1 at midnight UTC
signal[df.at_time('00:00').index] = 1
# Create a -1 right before the signal
signal[df.at_time('12:00').index] = -1 # for exiting 1 minute before midnight
df['signal'] = signal
# Convert to int
df['signal'] = df['signal'].astype(int)
df['signal'].sum()

0

## Create a helper function to keep your repo clean
Everytime you run a backtest it saves a file in your root. now it will save it in a folder called `html_files`

In [1]:
# This will be handy when we want to plot so it doesn't store all the html files in the root
def create_filename(stats_df):
    params = stats_df._strategy.__dict__['_params']
    filename = 'html_files/strat'
    # unpack the best params and add them to a filename
    for key, value in params.items():
        # keep the first 5 letters from each key
        key = key[:5]
        # just keep the first 4 digits from each value
        value= str(value)[:4]
        filename += f'_{key}_{value}_'
    filename += '.html'
    return filename

In [ ]:

short_df = df.loc['2023']

In [ ]:
# Create a benchmark strategy class that shows the impact of buying every day at midnight UTC and closing the trade at the end of the day
class Benchmark_Long(Strategy):
    def SIGNAL(self):
        return self.data.signal
    def init(self):
        super().init()
        self.signal = self.I(self.SIGNAL)
    
    def next(self):
        super().next()
        if self.signal == 1:
            self.buy()
        elif self.signal == -1:
            self.position.close()

## Run this if you want to see what a benchmark would be.
Notably if you went long the market every day and closed your position at the end of day, you would only have about a 50% win rate.

In [ ]:
# bt = Backtest(short_df, Benchmark_Long, cash=100_000, exclusive_orders=False)
# stats = bt.run()
# stats

Note ~ 50% of the days are positive and 50 are negative. So choosing an up or a down day is totally random. This should help with our testing.

# Create a money management strategy
We will have a quick profit rule, if the market moves in your favor you exit the position with a profit. Else, you fight like hell to manage the postion and end up with a profit.

### Let's also add the decay function
Below is an example of how the decay function will work within our class. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the custom decay function
def custom_decay_func(x, delay_period, upper_bound, lower_bound, delta_time):
    if x <= delay_period:
        return upper_bound
    elif delay_period < x < delay_period + delta_time:
        # Calculate the x value for the cos function
        transition_x = (x - delay_period) / delta_time * np.pi
        return (-np.cos(transition_x) + 1) / 2 * (lower_bound - upper_bound) + upper_bound
    else:
        return lower_bound

# Define the parameters
delay_period = 500  # 60 minutes
upper_bound_profit_target = 0.02 # Target Profit
lower_bound_loss_threshold = -0.05 # Worst acceptable loss
delta_time = 60*24*5  # how long the decay period lasts before leveling off

# Create an array of x values from 0 to 5000 minutes
x_values = np.linspace(0, 10000, 5001)

# Calculate the y values
y_values = [custom_decay_func(x, delay_period, upper_bound_profit_target, lower_bound_loss_threshold, delta_time) for x in x_values]

# Create the plot
plt.figure(figsize=(10, 3))
plt.plot(x_values, y_values)
plt.title('Decay Function Over Time')
plt.xlabel('Minutes Since First Trade')
plt.ylabel('Decayed Take Profit')
plt.grid(True)
plt.show()


### Build the strategy

In [ ]:
from tqdm.notebook import tqdm


class Underwater_w_decay_and_deleverage(Strategy):
    # All of the following variables can be used during optimization
    initial_position_size = 0.3
    percent_invested_threshold = 0.50
    atr_length = 14  # 14 days
    atr_multiplier = 0.5
    add_size = 0.30
    delay_period = 1000
    delta_time = 5000
    upper_bound_profit_target = 0.10
    lower_bound_loss_threshold = -0.05
    take_profit_loss_reduction = (
        -0.1
    )  # This is the amount that the take profit is reduced by if the position is highly leveraged and we wish to trim
    deleverage_pct = 0.30  # This is the amount that the position is reduced by if the position is highly leveraged and we wish to trim

    def SIGNAL(self):
        return self.data.signal

    def ATR(self, df, length):
        return df.ta.atr(length=length)

    def bars_since_first_trade(self, use_for_indexing=False):
        """
        Calculate the number of bars since the first trade was entered.
        If use for indexing is true then it will return 1 if there are no trades.
        This way you can retrieve the last element of a list by applying a - to the return value.
        eg. self.equity[-self.bars_since_first_trade(use_for_indexing=True):] gives you the array of your account's equity since you entered the first trade that is currently open
        """
        if len(self.trades) > 0 and self.trades[0].is_long:
            self.first_trade_entry_bar = self.trades[0].entry_bar
            bars_since_first_trade = len(self.data.Close) - self.first_trade_entry_bar
            return bars_since_first_trade
        elif use_for_indexing:
            return 1
        else:
            return 0

    def custom_decay_func(
        self,
        x,
        delay_period,
        upper_bound_profit_target,
        lower_bound_loss_threshold,
        delta_time,
    ):
        """
        This function is used to calculate the decayed take profit x represents the number of bars since the first trade was entered
        x is the number of bars since the first trade was entered
        delay_period is the number of bars to wait before starting to decay
        upper_bound_profit_target is the upper bound of the take profit
        lower_bound_loss_threshold is the lower bound of the take profit
        delta_time is the number of bars over which to transition from the upper bound to the lower bound
        """
        if x <= delay_period:
            return upper_bound_profit_target
        elif delay_period < x < delay_period + delta_time:
            # Calculate the x value for the cos function
            transition_x = (x - delay_period) / delta_time * np.pi
            # Calculate the decayed take profit
            return (-np.cos(transition_x) + 1) / 2 * (
                lower_bound_loss_threshold - upper_bound_profit_target
            ) + upper_bound_profit_target
        else:
            return lower_bound_loss_threshold

    def init(self):
        super().init()
        self.signal = self.I(self.SIGNAL)
        self.atr = self.I(self.ATR, self.data.df, self.atr_length)
        self.daily_atr = resample_apply("1D", self.ATR, self.data.df, length=14)
        count_NaN = len(self.atr) - len(
            pd.Series(self.atr).dropna()
        )  # This is used for the progress bar
        self.length_of_data = (
            len(self.data.Close) - count_NaN
        )  # This is used for the progress bar
        self.equity_during_trade = []  # Keeps a list for the equity during the trade

    def next(self):
        super().next()

        price = self.data.Close[-1]
        position_value = self.position.size * price
        percent_invested = (
            position_value / self.equity
        )  # this will come in handy if we decide to change behavior once XX% is invested
        atr_threshold_pct = (
            self.atr_multiplier * self.daily_atr[-1] / price
        )  # This is the ATR threshold times a multiplier calculated as a percentage of price
        bars_since_trade_open = self.bars_since_first_trade(use_for_indexing=True)
        highly_leveraged = percent_invested > self.percent_invested_threshold
        # Calculate the decayed take profit
        decayed_take_profit = self.custom_decay_func(
            bars_since_trade_open,
            self.delay_period,
            self.upper_bound_profit_target,
            self.lower_bound_loss_threshold,
            self.delta_time,
        )

        # Keep a running list of the self.equity values while we have a trade open
        if self.position.size > 0:
            self.equity_during_trade.append(self.equity)
        else:
            self.equity_during_trade = []

        low_point_in_trade = min(self.equity_during_trade, default=0)
        bounce_from_low_pct = (lambda x: (self.equity - x) / x if x != 0 else 0)(
            low_point_in_trade
        )  # The lambda function is used to avoid a divide by zero error

        if self.position.pl_pct > decayed_take_profit:
            self.position.close()

        # Deleverage if we are over a certain percent invested and the market is recovering from the low point
        elif (
            highly_leveraged and bounce_from_low_pct > 1.5 * atr_threshold_pct
        ):  # TODO: maybe make this a parameter we can tune
            # TODO: this is crude but it works for now
            # For now, just trim 20% of the position if we are within 5% of the take profit
            if (
                self.position.pl_pct
                > decayed_take_profit + self.take_profit_loss_reduction
            ):
                self.position.close(portion=self.deleverage_pct)

        elif self.position.size > 0 and self.position.pl_pct < -atr_threshold_pct:
            # Check to see if we are also down on our last trade
            if self.trades[-1].pl_pct < -atr_threshold_pct:
                self.buy(size=self.add_size)

        elif not self.position and self.signal == 1:
            self.buy(size=self.initial_position_size)

### Run the strategy

In [ ]:
start = '2022-05-01' # Note the strategy requires a warmup period for the ATR to calculate first trades begin after 14 days
end = '2022-12-03' # It will always close any open trades at the end of the backtest
bt = Backtest(df.loc[start:end], Underwater_w_decay_and_deleverage, cash=100_000_000, exclusive_orders=False, trade_on_close=True, margin=0.5)
stats = bt.run()
print(stats)

### Plot the strategy
Note you can add `resample=False` to see the actual details. This might be heavy on your browser though with anything more than 4-6 months of minute bars. 

In [ ]:
# bt.plot()
bt.plot(resample=False)

## Create an optimization
Note, this will take a while to run depending on how many parameters you want to optimize on.

I'm copying the class down here to remove the progress bars they conflict with the optimizer progress bars

In [ ]:
stats, heatmap = bt.optimize(
    # initial_position_size = np.arange(0.3, 0.6, 0.1).tolist(),
    # percent_invested_threshold = 0.7 
    # atr_length = 14 # 14 days
    # atr_multiplier = np.arange(0.6, 0.8, 0.1).tolist(),
    add_size = np.arange(0.10,0.30, 0.10).tolist(),
    # delay_period = 1000
    delta_time = np.arange(1000,5000,1000).tolist(),
    upper_bound_profit_target = np.arange(0.05, 0.10, 0.05).tolist(),
    lower_bound_loss_threshold = np.arange(0.05, -0.10, -0.05).tolist(),
    # take_profit_loss_reduction = -0.1 # This is the amount that the take profit is reduced by if the position is highly leveraged and we wish to trim
    # deleverage_pct = 0.5 # This is the amount that the position is reduced by if the position is highly leveraged and we wish to trim
    
    maximize='Equity Final [$]', # this can be any of the column names from the stats table the output of the backtest
    max_tries=200,
    random_state=0,
    return_heatmap=True)
heatmap.sort_values(ascending=False).iloc[:-5] # print the top 5 parameter sets

In [ ]:
# Plot the heatmap
from backtesting.lib import plot_heatmaps
plot_heatmaps(heatmap, agg='mean')


### Rerun the stats for the best strategy
`stats` is a dataframe with the results of the backtest or optimization but also with three objects, 
`_strategy`
`_equity_curve`
`_trades`

we can unpack the _strategy to ge the best params from the optimization and run that specific backtest.

In [ ]:
best_params = stats._strategy.__dict__["_params"] # This will print out all the parameters used for the best backtest

In [ ]:
bt.run(**best_params)

#### Now plot the results of the best strategy from the optimization

In [ ]:

bt.plot(resample=False)

If you want to work on a smaller sample period, just change `start` and `end` 

```python
start = '2021-01-01' # Note the strategy requires a warmup period for the ATR to calculate
end = '2022-3-31' # It will always close any open trades at the end of the backtest
bt = Backtest(df.loc[start:end], Underwater_w_decay_and_deleverage, cash=100_000_000, exclusive_orders=False, trade_on_close=True)
stats = bt.run()
print(stats)
```
If it is a small enough time period (few months or less) then you can run 

```python
bt.plot(resample=False)
```
Then you can see the detail of all the trades. 


In [ ]:
# stats['_equity_curve'] # To see the dataframe with the equity curve and the drawdown information

In [ ]:
# Or you can look at the full list
# stats['_trades']

In [ ]:

# Look at trades resampled on a daily basis
from backtesting.lib import TRADES_AGG
# stats['_trades'].resample('1D', on='ExitTime',
#                           label='right').agg(TRADES_AGG)

# Reverse the strategy for shorting

In [ ]:
from tqdm.notebook import tqdm


class SHORTING_Underwater_w_decay_and_deleverage(Strategy):
    # All of the following variables can be used during optimization
    initial_position_size = 0.3
    percent_invested_threshold = 0.50
    atr_length = 14  # 14 days
    atr_multiplier = 0.5
    add_size = 0.30
    delay_period = 1000
    delta_time = 5000
    upper_bound_profit_target = 0.10
    lower_bound_loss_threshold = -0.05
    take_profit_loss_reduction = (
        -0.1
    )  # This is the amount that the take profit is reduced by if the position is highly leveraged and we wish to trim
    deleverage_pct = 0.30  # This is the amount that the position is reduced by if the position is highly leveraged and we wish to trim

    def SIGNAL(self):
        return self.data.signal

    def ATR(self, df, length):
        return df.ta.atr(length=length)

    def bars_since_first_trade(self, use_for_indexing=False):
        """
        Calculate the number of bars since the first trade was entered.
        If use for indexing is true then it will return 1 if there are no trades.
        This way you can retrieve the last element of a list by applying a - to the return value.
        eg. self.equity[-self.bars_since_first_trade(use_for_indexing=True):] gives you the array of your account's equity since you entered the first trade that is currently open
        """
        if len(self.trades) > 0 and self.trades[0].is_short:
            self.first_trade_entry_bar = self.trades[0].entry_bar
            bars_since_first_trade = len(self.data.Close) - self.first_trade_entry_bar
            return bars_since_first_trade
        elif use_for_indexing:
            return 1
        else:
            return 0

    def custom_decay_func(
        self,
        x,
        delay_period,
        upper_bound_profit_target,
        lower_bound_loss_threshold,
        delta_time,
    ):
        """
        This function is used to calculate the decayed take profit x represents the number of bars since the first trade was entered
        x is the number of bars since the first trade was entered
        delay_period is the number of bars to wait before starting to decay
        upper_bound_profit_target is the upper bound of the take profit
        lower_bound_loss_threshold is the lower bound of the take profit
        delta_time is the number of bars over which to transition from the upper bound to the lower bound
        """
        if x <= delay_period:
            return upper_bound_profit_target
        elif delay_period < x < delay_period + delta_time:
            # Calculate the x value for the cos function
            transition_x = (x - delay_period) / delta_time * np.pi
            # Calculate the decayed take profit
            return (-np.cos(transition_x) + 1) / 2 * (
                lower_bound_loss_threshold - upper_bound_profit_target
            ) + upper_bound_profit_target
        else:
            return lower_bound_loss_threshold

    def init(self):
        super().init()
        self.signal = self.I(self.SIGNAL)
        self.atr = self.I(self.ATR, self.data.df, self.atr_length)
        self.daily_atr = resample_apply("1D", self.ATR, self.data.df, length=14)
        count_NaN = len(self.atr) - len(
            pd.Series(self.atr).dropna()
        )  # This is used for the progress bar
        self.length_of_data = (
            len(self.data.Close) - count_NaN
        )  # This is used for the progress bar
        self.equity_during_trade = []  # Keeps a list for the equity during the trade

    def next(self):
        super().next()

        price = self.data.Close[-1]
        position_value = self.position.size * price
        percent_invested = (
            position_value / self.equity
        )  # this will come in handy if we decide to change behavior once XX% is invested
        atr_threshold_pct = (
            self.atr_multiplier * self.daily_atr[-1] / price
        )  # This is the ATR threshold times a multiplier calculated as a percentage of price
        bars_since_trade_open = self.bars_since_first_trade(use_for_indexing=True)
        highly_leveraged = percent_invested > self.percent_invested_threshold
        # Calculate the decayed take profit
        decayed_take_profit = self.custom_decay_func(
            bars_since_trade_open,
            self.delay_period,
            self.upper_bound_profit_target,
            self.lower_bound_loss_threshold,
            self.delta_time,
        )

        # Keep a running list of the self.equity values while we have a trade open
        if self.position.size < 0:
            self.equity_during_trade.append(self.equity)
        else:
            self.equity_during_trade = []

        low_point_in_trade = min(self.equity_during_trade, default=0)
        bounce_from_low_pct = (lambda x: (self.equity - x) / x if x != 0 else 0)(
            low_point_in_trade
        )  # The lambda function is used to avoid a divide by zero error

        if self.position.pl_pct > decayed_take_profit:
            self.position.close()

        # Deleverage if we are over a certain percent invested and the market is recovering from the low point
        elif (
            highly_leveraged and bounce_from_low_pct > 1.5 * atr_threshold_pct
        ):  # TODO: maybe make this a parameter we can tune
            # TODO: this is crude but it works for now
            # For now, just trim 20% of the position if we are within 5% of the take profit
            if (
                self.position.pl_pct
                > decayed_take_profit + self.take_profit_loss_reduction
            ):
                self.position.close(portion=self.deleverage_pct)

        elif self.position.size < 0 and self.position.pl_pct < -atr_threshold_pct:
            # Check to see if we are also down on our last trade
            if self.trades[-1].pl_pct < -atr_threshold_pct:
                self.sell(size=self.add_size)

        elif not self.position and self.signal == 1:
            self.sell(size=self.initial_position_size)

In [ ]:
start = '2022-05-01' # Note the strategy requires a warmup period for the ATR to calculate first trades begin after 14 days
end = '2022-12-03' # It will always close any open trades at the end of the backtest
bt = Backtest(df.loc[start:end], SHORTING_Underwater_w_decay_and_deleverage, cash=100_000_000, exclusive_orders=False, trade_on_close=True, margin=0.5)
stats = bt.run()
print(stats)
bt.plot(resample=False)

# Strategy combining Long and Short signals
### WIP 
Our signals are 1 at 00:01 every day and -1 at 12:00 every day. If we are not in a trade, then whichever signal comes first it will work it. 
@hoang, if you can read in the lstm buy signals as a 1 and open sell signals as a -1 then I think this will all work right away. We will have to think about duplicated signals but I don't think that is a problem. 

In [7]:
class LONG_SHORT_Underwater_w_decay_and_deleverage(Strategy):
    # All of the following variables can be used during optimization
    initial_position_size = 0.3
    percent_invested_threshold = 0.3
    atr_length = 14  # 14 days
    atr_multiplier = 0.5
    add_size = 0.1
    delay_period = 1000
    delta_time = 1000
    upper_bound_profit_target = 0.05
    lower_bound_loss_threshold = 0.02
    take_profit_loss_reduction = -0.1 # amount take profit is reduced by if the position is highly leveraged and we wish to trim
    deleverage_pct = 0.30  # This is the amount that the position is reduced by if the position is highly leveraged and we wish to trim
    bounce_multiplier = 1.5  # multiply the ATR Treshold by this amount to get the bounce threshold we are looking to bounce off a low "max pain" point before we reduce our position
    max_loss_threshold = -0.05 # if the position is down this much, and we are fully invested we will reduce our positio
    max_hold_length = None # Number of minutes to hold a position before closing it None if you want to exit naturally
    
    def SIGNAL(self):
        return self.data.signal

    def ATR(self, df, length):
        return df.ta.atr(length=length)

    def bars_since_first_trade(self, use_for_indexing=False):
        """
        Calculate the number of bars since the first trade was entered.
        If use for indexing is true then it will return 1 if there are no trades.
        This way you can retrieve the last element of a list by applying a - to the return value.
        eg. self.equity[-self.bars_since_first_trade(use_for_indexing=True):] gives you the array of your account's equity since you entered the first trade that is currently open
        """
        if len(self.trades) > 0:
            self.first_trade_entry_bar = self.trades[0].entry_bar
            bars_since_first_trade = len(self.data.Close) - self.first_trade_entry_bar
            return bars_since_first_trade
        elif use_for_indexing:
            return 1
        else:
            return 0

    def custom_decay_func(
        self,
        x,
        delay_period,
        upper_bound_profit_target,
        lower_bound_loss_threshold,
        delta_time,
    ):
        """
        This function is used to calculate the decayed take profit x represents the number of bars since the first trade was entered
        x is the number of bars since the first trade was entered
        delay_period is the number of bars to wait before starting to decay
        upper_bound_profit_target is the upper bound of the take profit
        lower_bound_loss_threshold is the lower bound of the take profit
        delta_time is the number of bars over which to transition from the upper bound to the lower bound
        """
        if x <= delay_period:
            return upper_bound_profit_target
        elif delay_period < x < delay_period + delta_time:
            # Calculate the x value for the cos function
            transition_x = (x - delay_period) / delta_time * np.pi
            # Calculate the decayed take profit
            return (-np.cos(transition_x) + 1) / 2 * (
                lower_bound_loss_threshold - upper_bound_profit_target
            ) + upper_bound_profit_target
        else:
            return lower_bound_loss_threshold

    def init(self):
        super().init()
        self.signal = self.I(self.SIGNAL)
        self.atr = self.I(self.ATR, self.data.df, self.atr_length)
        self.daily_atr = resample_apply("1D", self.ATR, self.data.df, length=14)
        count_NaN = len(self.atr) - len(
            pd.Series(self.atr).dropna()
        )  # This is used for the progress bar
        self.length_of_data = (
            len(self.data.Close) - count_NaN
        )  # This is used for the progress bar
        self.equity_during_trade = []  # Keeps a list for the equity during the trade
        self.long_short_flag = (
            None  # This is used to keep track of whether we are long or short
        )
        self.price_at_last_trim = (
            0  # This is used to keep track of the price at the last trim
        )

    def next(self):
        super().next()

        price = self.data.Close[-1]
        position_value = self.position.size * price
        percent_invested = (
            position_value / self.equity
        )  # this will come in handy if we decide to change behavior once XX% is invested
        atr_threshold_pct = (
            self.atr_multiplier * self.daily_atr[-1] / price
        )  # This is the ATR threshold times a multiplier calculated as a percentage of price
        bars_since_trade_open = self.bars_since_first_trade(use_for_indexing=True)
        highly_leveraged = abs(percent_invested) > self.percent_invested_threshold
        # Calculate the decayed take profit
        decayed_take_profit = self.custom_decay_func(
            bars_since_trade_open,
            self.delay_period,
            self.upper_bound_profit_target,
            self.lower_bound_loss_threshold,
            self.delta_time,
        )

        # Calculate the lowest the equity ever got since we have been in a trade
        if self.position:  
            self.lowest_equity_during_trade = min(self.lowest_equity_during_trade, self.equity)
        else:
            self.lowest_equity_during_trade = float('inf')  # Reset to a very high number when not in a position

        low_point_in_trade = min(self.equity_during_trade, default=0)

        # Calculate the percentage bounce from the low point in the trade this is based on equity so works for long or short
        bounce_from_low_pct = (lambda x: (self.equity - x) / x if x != 0 else 0)(
            low_point_in_trade
        )  # The lambda function is used to avoid a divide by zero error

        # Opening a Trade on a signal from the LSTM
        if not self.position:
            if self.signal == 1:
                self.buy(size=self.initial_position_size)
                self.long_short_flag = 1
            elif self.signal == -1:
                self.sell(size=self.initial_position_size)
                self.long_short_flag = -1
        
        # If we are in a short trade and the account is fully invested and the loss is greater than the max loss threshold then close the trade
        if self.long_short_flag == -1 and (abs(percent_invested) > 1 and self.position.pl_pct < self.max_loss_threshold):
            self.position.close() # If you want a partial close add `portion=self.deleverage_pct`
        
        # If we are in a trade and it meets our profit criteria then close the trade
        if self.position.pl_pct > decayed_take_profit:
            self.position.close()
            self.price_at_last_trim = 0
            # print(f'Closing at {price} at {self.data.index[-1]} Position PNL is {self.position.pl}')

        # If we are in a trade and it meets our loss criteria then close a portion of the trade on a bounce from the low point
        if self.position and self.position.pl_pct < -atr_threshold_pct:
            # Check to see if we are also down on our last trade
            if self.trades[-1].pl_pct < -atr_threshold_pct:
                if self.long_short_flag == 1:
                    self.buy(size=self.add_size)
                elif self.long_short_flag == -1:
                    self.sell(size=self.add_size)
        
        # If we are totally upside down then reduce the position size
        if self.position and self.position.pl_pct < decayed_take_profit + self.take_profit_loss_reduction:
            if self.trades[-1].pl_pct < -atr_threshold_pct: # if we are down more than 1 ATR threshold then reduce the position size and take a tiny loss
                self.position.close(portion=self.deleverage_pct)
                # print(f'Closing at {price} at {self.data.index[-1]} Position PNL is {self.position.pl}')
        
        # Deleverage if we are over a certain percent invested and the market is recovering from the low point
        if (
            highly_leveraged
            and bounce_from_low_pct > self.bounce_multiplier * atr_threshold_pct
        ):

            if (self.position.pl_pct > decayed_take_profit + self.take_profit_loss_reduction):
                if (self.long_short_flag == 1) and (self.price_at_last_trim == 0 or price > self.price_at_last_trim *(1 + 2 * atr_threshold_pct)):
                    self.position.close(portion=self.deleverage_pct)
                    # Keep track of the price to avoid trimming too often
                    self.price_at_last_trim = price 
                    # print(f'Deleveraging at {price} at {self.data.index[-1]} Position PNL is {self.position.pl}')
                elif (self.long_short_flag) == -1 and (self.price_at_last_trim == 0 or price < self.price_at_last_trim *(1 - 0.5 * atr_threshold_pct)):
                    self.position.close(portion=self.deleverage_pct)
                    # Keep track of the price to avoid trimming too often
                    self.price_at_last_trim = price 
                    # print(f'Deleveraging at {price} at {self.data.index[-1]} Position PNL is {self.position.pl}')



start = "2022-01-01"  # Note the strategy requires a warmup period for the ATR to calculate first trades begin after 14 days
end = "2023-06-03"  # It will always close any open trades at the end of the backtest
bt = Backtest(
    df.loc[start:end],
    LONG_SHORT_Underwater_w_decay_and_deleverage,
    cash=100_000_000,
    exclusive_orders=False,
    trade_on_close=True,
    margin=1,
)
stats = bt.run()
filename = create_filename(stats)
print(stats)
bt.plot(resample=False, filename=filename)

Start                     2022-01-01 00:00...
End                       2023-06-03 23:12...
Duration                    518 days 23:12:00
Exposure Time [%]                   91.453648
Equity Final [$]                  111616681.5
Equity Peak [$]                   111841269.5
Return [%]                          11.616682
Buy & Hold Return [%]              -41.491126
Return (Ann.) [%]                    8.035556
Volatility (Ann.) [%]               23.318242
Sharpe Ratio                         0.344604
Sortino Ratio                        0.490093
Calmar Ratio                         0.388506
Max. Drawdown [%]                  -20.683231
Avg. Drawdown [%]                   -0.421241
Max. Drawdown Duration      406 days 05:49:00
Avg. Drawdown Duration        3 days 01:40:00
# Trades                                  898
Win Rate [%]                        23.385301
Best Trade [%]                      12.810546
Worst Trade [%]                    -43.835413
Avg. Trade [%]                    

GridPlot(id='p1721', ...)

## Now let's optimize the long short strategy
Note, this will take a while to run depending on how many parameters you want to optimize on.

I'm copying the class down here to remove the progress bars they conflict with the optimizer progress bars

In [ ]:
stats, heatmap = bt.optimize(
    # initial_position_size = np.arange(0.3, 0.6, 0.1).tolist(),
    # percent_invested_threshold = 0.7 
    # atr_length = np.arange(7,37,10).to_list(), # 14 days
    # atr_multiplier = np.arange(0.6, 0.8, 0.1).tolist(),
    add_size = np.arange(0.10,0.30, 0.10).tolist(),
    # delay_period = np.arange(250,1500,500).tolist(),
    delta_time = np.arange(1000,5000,1000).tolist(),
    upper_bound_profit_target = np.arange(0.05, 0.10, 0.05).tolist(),
    lower_bound_loss_threshold = np.arange(0.05, -0.10, -0.05).tolist(),
    take_profit_loss_reduction = np.arange(-0.15, -0.05, 0.05).tolist(), # This is the amount that the take profit is reduced by if the position is highly leveraged and we wish to trim
    # deleverage_pct = 0.5 # This is the amount that the position is reduced by if the position is highly leveraged and we wish to trim
    
    maximize='Equity Final [$]', # this can be any of the column names from the stats table the output of the backtest
    max_tries=200,
    random_state=0,
    return_heatmap=True)
best_params = stats._strategy.__dict__["_params"] # This will print out all the parameters used for the best backtest
print(f'Best Parameters: {best_params}')
heatmap.sort_values(ascending=False).iloc[:-5] # print the top 5 parameter sets

In [ ]:
# Plot the heatmap
from backtesting.lib import plot_heatmaps
plot_heatmaps(heatmap, agg='mean')


### Rerun the stats for the best strategy


In [ ]:
bt.run(**best_params)
bt.plot(resample=False)

# Thoughts for additional improvements
- add a percent_of_equity method that would take total risk into account.
- perhaps reduce risk when leverage is high and we are recovering from a big underwater period.
- Consider a dynamic take profit target and lower loss limit ie. (2*ATR for upper) and (-3*ATR for lower)
- Weekly or monthly PnL targets to reduce quantity of time we are in market

# Add in our signals
Run the strategies long and short based on our own AI buy and sell signals. Use it as an opportunity to improve the results of our existing AI models.
#### Read in a dataframe of our signals
prep the dataframe so it has the `Close` price, the `signal` column. 
For this we can create an `entries` column and an `exits` column.

| trigger (from CSV)  | entries          | exits            | signal |
| --------            | --------         | --------         | --     |
| 1 day passed        | None             | 1 day passed     | 0      |
| lstm_open-long      | lstm_open-long   | None             | 1      |
| lstm_open-short     | lstm_open-short  | None             | -1     |
| NA                  | None             | None             | 0      |
| stop loss           | None             | stop loss        | 0      |
| take profit         | None             | take profit      | 0      |

                      

We can then tweak our code above to read from the `entries` column instead of the `signal` column. We will be ignoring the LSTM exits for now because the backtest strategy takes over once we are in a trade. Perhaps this is something we will change in the future but for now we can keep it as is.


# Import the trades dataframe and manipulate it.

## Step 1 - Read in the CSV files

In [13]:
from typing import List
lstm_csv_files = ["test_data/2dc0f4b2_minutely.csv",
"test_data/3dca0a12_minutely.csv",
"test_data/9295144c_minutely.csv",
"test_data/b2d47ab1_minutely.csv",
"test_data/b9962cd1_minutely.csv"]

def _read_lstm_files() -> List:
  lstm_dfs = []

  for file in lstm_csv_files:
    lstm_dfs.append(pd.read_csv(file))

  return lstm_dfs
    

## Define constants and mappings

In [14]:
from dataclasses import dataclass
from typing import Optional
from collections import defaultdict

@dataclass
class BackTestColumnValues:
  entries : Optional[str]
  exits   : Optional[str]
  signal  : int

LSTM_TRIGGER_MAP = defaultdict(
  lambda            : BackTestColumnValues(None             , None            , 0 ), 
{
  "1 day passed"    : BackTestColumnValues(None             , "1 day passed"  , 0 ),
  "lstm_open-long"  : BackTestColumnValues("lstm_open-long" , None            , 1 ),
  "lstm_open-short" : BackTestColumnValues("lstm_open-short", None            , -1),  
  "N/A"             : BackTestColumnValues(None             , None            , 0 ),
  "stop loss"       : BackTestColumnValues(None             , "stop loss"     , 0 ),
  "take profit"     : BackTestColumnValues(None             , "take profile"  , 0 ),
})



## Define the transformation 

In [15]:
def _map_lstm_trigger_to_signal(trigger: str) -> Optional[int]:
  return LSTM_TRIGGER_MAP[trigger].signal


def _map_lstm_trigger_to_entries(trigger: str) -> Optional[str]:
  return LSTM_TRIGGER_MAP[trigger].entries


def _map_lstm_trigger_to_exits(trigger: str) -> Optional[str]:
  return LSTM_TRIGGER_MAP[trigger].exits


def _transform_lstm_dfs(lstm_dfs: List):
  for lstm_df in lstm_dfs:    
    lstm_df.rename(columns={'price': 'Close'}, inplace=True)

    lstm_df['trigger']  = lstm_df['trigger'].astype(str)
    lstm_df['entries']  = lstm_df['trigger'].apply(_map_lstm_trigger_to_entries)
    lstm_df['exits']    = lstm_df['trigger'].apply(_map_lstm_trigger_to_exits)
    lstm_df['signal']   = lstm_df['trigger'].apply(_map_lstm_trigger_to_signal)    
    lstm_df['time']     = pd.to_datetime(lstm_df['time'], unit='s').dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

## The starting point of backtesting LSTM CSV files

In [16]:
lstm_dfs = _read_lstm_files()
_transform_lstm_dfs(lstm_dfs)
test_df = lstm_dfs[0]
test_df.index = pd.to_datetime(test_df['time'])

    

,time,Close,trigger,entries,exits,signal
0,2021-01-09T00:01:00.000000Z,40667.33,nan,None,None,0
1,2021-01-09T00:02:00.000000Z,40722.11,nan,None,None,0
2,2021-01-09T00:03:00.000000Z,40873.86,nan,None,None,0
3,2021-01-09T00:04:00.000000Z,40921.72,nan,None,None,0
4,2021-01-09T00:05:00.000000Z,40954.88,nan,None,None,0
...,...,...,...,...,...,...
707035,2022-05-14T23:56:00.000000Z,30134.50,nan,None,None,0
707036,2022-05-14T23:57:00.000000Z,30157.90,nan,None,None,0
707037,2022-05-14T23:58:00.000000Z,30082.30,nan,None,None,0
707038,2022-05-14T23:59:00.000000Z,30070.70,nan,None,None,0


# Run the backtest on the lstm entries from the new dataframe

# Optimize the parameters

# Test on different periods of time Possibly set up a cross validation
Nice youtube video here https://www.youtube.com/watch?v=9m987swadQU&t=2154s&ab_channel=ChadThackray